In [1]:
# Resume parsing
import unicodedata
import fitz  # PyMuPDF


# Open the PDF file
with fitz.open('data/JasmineBellResume.pdf') as doc:
    text = ""
    for page in doc:
        text += page.get_text()

def cleanup_unicode(text):
    corrected_chars = []
    for char in text:
        corrected_char = unicodedata.normalize("NFKC", char)
        corrected_chars.append(corrected_char)
    return "".join(corrected_chars)


text = cleanup_unicode(text)


In [2]:
print(text)

JB
JASMINE BELL
Full Stack Software Developer
407.601.0755
jasmine.bell23@gmail.com
front-median.net
Cathedral City, Grenada
SUMMARY
Experienced Full Stack Software Developer with a strong background in JavaScript, .NET Framework, and ReactJS. I have 
successfully delivered high-quality software solutions and optimized system performance resulting in notable 
achievements, such as a 30% reduction in response time and receiving an Innovation Award. I am an effective problem 
solver, collaborate well in cross-functional teams, and thrive in fast-paced environments. With a Bachelor's degree in 
Computer Science and a passion for innovation, I am excited to contribute to a company's mission and be part of its 
sustained growth and creative work.
STRENGTHS
Problem Solving
Strong problem-solving skills demonstrated through 
successfully resolving complex technical issues 
resulting in improved system performance.
Collaboration
Effective team player with excellent collaboration 
skills, contr

In [3]:
from dataclasses import dataclass
from llm_core.assistants import LLaMACPPAssistant


@dataclass
class JobExperience:
    job_title: str
    company: str
    dates: str
    location: str
    software_tools: list[str]

@dataclass
class Resume:
    first_name: str
    last_name: str
    email: str
    experiences: list[JobExperience]
    

@dataclass
class ResumeParser:
    system_prompt = """You are an HR assistant"""
    
    prompt = """
    Resume:
    {resume}

    Carefully parse the resume
    """

    resume: Resume

    @classmethod
    def parse(cls, resume):
        # You need to download a GGUF model and copy it into ~/.cache/py-llm-core/models/
        # See README.md
        with LLaMACPPAssistant(cls, model='mistral-v0.1-q4') as assistant:
            return assistant.process(resume=resume)

In [4]:
response = ResumeParser.parse(text)

In [5]:
print(response.resume)

Resume(first_name='Jasmine', last_name='Bell', email='jasmine.bell23@gmail.com', experiences=[JobExperience(job_title='Full Stack Software Developer', company='Tech Solutions Ltd', dates='2022-Present', location='London, UK', software_tools=['ReactJS', 'Redux', 'Thunk', '.NET Core', '.NET WCF', 'SQL Server', 'GitFlow', 'HTML', 'CSS', 'JavaScript', 'SQL stored procedures', 'RESTful web services', 'Continuous Integration', 'ATDD', 'TDD']), JobExperience(job_title='Software Engineer', company='Innovative Tech Solutions', dates='2017-2022', location='London, UK', software_tools=['.NET Framework', 'ASP.NET MVC', 'Razor', 'SQL Server', 'JavaScript', 'jQuery', 'responsive web designs', 'code reviews']), JobExperience(job_title='Junior Software Developer', company='Tech Innovators Inc', dates='2015-2017', location='London, UK', software_tools=['C#.NET', 'SQL Server', 'agile development methodologies', 'web forms', 'user controls', 'RESTful APIs', 'third-party libraries'])])


In [6]:
for job_xp in response.resume.experiences:
    print(job_xp.job_title)
    print(job_xp.company)
    print(job_xp.software_tools)
    print('*'*80)

Full Stack Software Developer
Tech Solutions Ltd
['ReactJS', 'Redux', 'Thunk', '.NET Core', '.NET WCF', 'SQL Server', 'GitFlow', 'HTML', 'CSS', 'JavaScript', 'SQL stored procedures', 'RESTful web services', 'Continuous Integration', 'ATDD', 'TDD']
********************************************************************************
Software Engineer
Innovative Tech Solutions
['.NET Framework', 'ASP.NET MVC', 'Razor', 'SQL Server', 'JavaScript', 'jQuery', 'responsive web designs', 'code reviews']
********************************************************************************
Junior Software Developer
Tech Innovators Inc
['C#.NET', 'SQL Server', 'agile development methodologies', 'web forms', 'user controls', 'RESTful APIs', 'third-party libraries']
********************************************************************************


In [3]:
# Use NuExtract

from dataclasses import dataclass, field
from llm_core.parsers import NuExtractParser


@dataclass
class JobExperience:
    job_title: str = ""
    company: str = ""
    dates: str = ""
    location: str = ""
    software_tools: list[str] = field(default_factory=lambda: [""])

@dataclass
class Resume:
    first_name: str = ""
    last_name: str = ""
    email: str = ""
    experiences: list[JobExperience] = field(default_factory=lambda: [JobExperience()])
    

@dataclass
class ResumeParser:
    resume: Resume = field(default_factory=lambda: Resume())

    @classmethod
    def parse(cls, resume):
        with NuExtractParser(cls, model="nuextract") as parser:
            response = parser.parse(resume)
            print(response)

    

In [4]:
response = ResumeParser.parse(text)
print(response)

ResumeParser(resume=Resume(first_name='JASMINE', last_name='BELL', email='jasmine.bell23@gmail.com', experiences=[JobExperience(job_title='Full Stack Software Developer', company='Tech Solutions Ltd', dates='2022 - Present', location='London, UK', software_tools=['ReactJS', 'Redux', 'Thunk']), JobExperience(job_title='Software Engineer', company='Innovative Tech Solutions', dates='2017 - 2022', location='London, UK', software_tools=['.NET Framework', 'ASP.NET MVC', 'Razor']), JobExperience(job_title='Junior Software Developer', company='Tech Innovators Inc', dates='2015 - 2017', location='London, UK', software_tools=['C#.NET', 'SQL Server'])]))
None
